# Day 1 실습 4: Baseline vs LoRA 모델 비교

## 🎯 이 노트북의 목적

03번에서 학습한 LoRA 모델이 **실제로 성능이 향상되었는지** 검증합니다.

### 비교 대상

- **Baseline**: 파인튜닝 전 원본 EXAONE-3.5 모델
- **LoRA**: 한국어 요약 데이터로 파인튜닝한 모델

### 평가 방법

1. 같은 테스트 데이터로 두 모델 모두 요약 생성
2. ROUGE-1, 임베딩 유사도로 정답과 비교
3. 평균 점수 비교

## 0. 패키지 설치

Colab이라면 아래를 실행하고 **런타임 재시작**하세요.

```python
!pip install -q --upgrade bitsandbytes accelerate sentence-transformers evaluate rouge-score
```

In [ ]:
!pip install -q --upgrade bitsandbytes accelerate sentence-transformers evaluate rouge-score

# 설치 후 런타임 재시작 필요!

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

print("✅ 라이브러리 로드 완료")

## 1. 모델 로드

In [ ]:
BASE_MODEL = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
LOCAL_ADAPTER = "./exaone_summary_lora_final"  # 03번에서 저장한 로컬 경로
HF_ADAPTER = "ryanu/exaone-summary-lora"  # 허깅페이스 업로드 경로

print("📥 토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
)

print("📥 Baseline 모델 로드 중...")
baseline_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=quant_config,
)

print("📥 LoRA 모델 로드 중...")
if os.path.isdir(LOCAL_ADAPTER):
    ADAPTER_DIR = LOCAL_ADAPTER
    print(f"   ✅ 로컬 어댑터 사용: {ADAPTER_DIR}")
else:
    ADAPTER_DIR = HF_ADAPTER
    print(f"   ✅ 허깅페이스 어댑터 사용: {ADAPTER_DIR}")

lora_model = PeftModel.from_pretrained(baseline_model, ADAPTER_DIR)

print("\n✅ 모델 로드 완료")

## 2. 평가 데이터 로드

In [ ]:
from datasets import load_dataset

print("📥 평가 데이터 로드 중...")
test_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split="test")

# 토큰 길이 필터링 (03번과 동일)
def is_valid_sample(example):
    article = str(example.get("document", ""))
    summary = str(example.get("summary", ""))
    estimated_tokens = len(article) / 1.5 + len(summary) / 1.5
    return estimated_tokens < 512 * 0.8

print(f"📊 원본 데이터: {len(test_dataset):,}개")

test_dataset = test_dataset.filter(is_valid_sample)

print(f"✂️ 필터링 후: {len(test_dataset):,}개")

MAX_EVAL_SAMPLES = 20
test_samples = test_dataset.select(range(min(MAX_EVAL_SAMPLES, len(test_dataset))))

print(f"✅ 평가 샘플: {len(test_samples)}개")


## 3. 응답 생성 함수

In [ ]:
def generate_summary(model, tokenizer, article, max_new_tokens=60):
    """요약 생성 함수"""
    prompt = article + "요약:"
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 프롬프트 제거
    if "요약:" in generated:
        return generated.split("요약:")[-1].strip()
    return generated[len(prompt):].strip()

print("✅ 생성 함수 정의 완료")

## 4. 평가 지표 (ROUGE-1, 임베딩 유사도)

### 💡 평가 지표 설명

| 지표 | 설명 | 해석 |
|------|------|------|
| **ROUGE-1** | 단어 단위 겹침 비율 (0~1) | 높을수록 정답과 비슷한 단어 사용 |
| **임베딩 유사도** | 의미적 유사도 (cosine, -1~1) | 높을수록 의미가 비슷함 |

### 왜 2개를 사용?

- ROUGE-1: 단어가 정확히 겹치는지 확인
- 임베딩 유사도: 다른 표현이어도 의미가 같으면 높은 점수

In [ ]:
from sentence_transformers import SentenceTransformer
import evaluate
import numpy as np

print("📥 평가 도구 로드 중...")
rouge = evaluate.load("rouge")
embed_model = SentenceTransformer("BAAI/bge-m3", device="cuda" if torch.cuda.is_available() else "cpu")

def compute_rouge1(reference, candidate):
    try:
        return rouge.compute(predictions=[candidate], references=[reference])["rouge1"]
    except:
        return 0.0

def compute_embedding_similarity(reference, candidate):
    try:
        if not reference.strip() or not candidate.strip():
            return 0.0
        embeddings = embed_model.encode([reference, candidate], convert_to_numpy=True, normalize_embeddings=True)
        return float(np.dot(embeddings[0], embeddings[1]))
    except:
        return 0.0

print("✅ 평가 도구 준비 완료")

## 5. 평가 실행

In [ ]:
results = []

print("🔍 평가 시작...\n")

for idx, item in enumerate(test_samples):
    article = item["document"][:500]  # 너무 길면 앞부분만
    reference = item["summary"]
    
    # Baseline 생성
    baseline_output = generate_summary(baseline_model, tokenizer, article)
    
    # LoRA 생성
    lora_output = generate_summary(lora_model, tokenizer, article)
    
    # 점수 계산
    baseline_rouge = compute_rouge1(reference, baseline_output)
    lora_rouge = compute_rouge1(reference, lora_output)
    baseline_emb = compute_embedding_similarity(reference, baseline_output)
    lora_emb = compute_embedding_similarity(reference, lora_output)
    
    results.append({
        "article": article[:100] + "...",
        "reference": reference,
        "baseline": baseline_output,
        "lora": lora_output,
        "baseline_rouge1": baseline_rouge,
        "lora_rouge1": lora_rouge,
        "baseline_embedding": baseline_emb,
        "lora_embedding": lora_emb,
    })
    
    if (idx + 1) % 5 == 0:
        print(f"   {idx + 1}/{len(test_samples)} 완료")

results_df = pd.DataFrame(results).round(3)

print("\n✅ 평가 완료!\n")
display(results_df)

## 6. 평균 점수 비교

In [ ]:
summary = pd.DataFrame({
    "Metric": ["ROUGE-1", "Embedding Cosine"],
    "Baseline": [
        results_df["baseline_rouge1"].mean(),
        results_df["baseline_embedding"].mean(),
    ],
    "LoRA": [
        results_df["lora_rouge1"].mean(),
        results_df["lora_embedding"].mean(),
    ],
}).round(3)

print("\n" + "="*60)
print("📊 평균 점수 비교")
print("="*60)
display(summary)

print("\n개선도:")
for metric, delta in zip(summary["Metric"], summary["LoRA"] - summary["Baseline"]):
    arrow = "🔼" if delta > 0 else ("🔽" if delta < 0 else "➖")
    print(f"{arrow} {metric}: {delta:+.3f}")

## 7. 결과 해석

### 📊 점수 해석 기준

| 지표 | Baseline | LoRA 기대치 | 의미 |
|------|----------|-------------|------|
| **ROUGE-1** | 0.1~0.3 | +0.05~0.15 | 단어 겹침 개선 |
| **임베딩 유사도** | 0.5~0.7 | +0.05~0.10 | 의미적 유사도 개선 |

### ✅ 성공 기준

- LoRA가 Baseline보다 **모든 지표에서 높으면 성공**
- ROUGE-1이 0.1 이상 차이나면 **큰 개선**
- 임베딩 유사도가 0.05 이상 차이나면 **의미적으로 향상**

### ⚠️ 만약 LoRA가 더 낮다면?

- 학습 데이터가 너무 적음 (epoch 늘리기)
- LoRA rank가 너무 낮음 (r=8 → r=16)
- 학습률이 너무 높거나 낮음

### 다음 단계

- **05번 노트북**: 여러 모델로 LoRA 비교 실험